## 1. Hashing
-----------------------------------------------------------------------------------

In [1]:
import numpy as np
import random 

In [2]:
def hash_function(string):
    m=2**32
    hash_= 7
    for i in range(len(string)):
        x = encoding(string[i])
        hash_=((hash_*23))+x
    hash_=hash_%m #to avoid overflow
    binary = bin(hash_)[2:]
    str_binary = str(binary)
    return str_binary

On each char of the input string, such as a line of the file,we apply the ecoding before computing the hashing.
With this function we convert each item of the string into an ASCII char.

In [3]:
def encoding(string):
    s=0
    for char in string:
        s+=ord(char) 
    return s

## Hyperloglog
Now we use the Hyperloglog structure to store our binary codes. To remove the effects of outliers, and to lower the variance, HyperLogLog splits the input into several subsets called “buckets”. Then for each buckets the estimates are combined using an harmonic mean to deduce an estimate of the cardinality for the entire set.


-----------------------------------------------------------------------------------------------------


## Add item to the buckets: 
One of the supported operation on this data structure is to **add** items.
It consists of computing the hash each line in the file with the hash_function define above, getting the first n bits (where n is ${\textstyle \log _{2}(m)}$) to obtain the address of the bucket to modify. With the remaining bits we compute ${\textstyle \rho (w)}$ which returns the position of the leftmost 1, computed with the below function **max_zeros**.

The new value of the register will be the maximum between the current value of the register and ${\textstyle \rho (w)}$.

In [4]:
def max_zeros(string):
    maximum=0
    tmp=0
    for i in range(len(string)):
        if string[i]=="0":
            tmp+=1
        else:
            maximum=tmp
            break
    return maximum

Assuming that the output of our hash is for example hash(input)=101101110110111001100011010110100. We use the leftmost n=13 bits to find the bucket index, so **10110111011011**1001100011010110100 tells us which bucket to update. 

```j = int(digit[:n],2)``` 


From the remaining, 10110111011011**1001100011010110100** we extract the longest run of consecutive 0s from the rightmost bits and they are used to find the bucket number. 
``` 
w=digit[n:]  
buckets[j]=max(buckets[j],max_zeros(w))
```

In [5]:
def hyperloglog(file):
    length_file = 139000000     
    f = open(file, "r") #open the file
    n=13
    m = 2**n
    #initialize the buckets
    buckets = [0 for x in range (m)]
    #for each rows in the file
    for row in range(0,length_file):
        #read one line at time
        line = f.readline(row).strip()
        
        #compute the hash function on it and save the value in digit
        digit = hash_function(line) 
        
        #filling with zeros in order to obtain digit of 32 bit
        if len(digit)<32: 
            digit = '0'*(32-len(digit))+digit
             
        #add items to the buckets
        j = int(digit[:n],2) 
        w=digit[n:] 
        buckets[j]=max(buckets[j],max_zeros(w))
           
    #computing cardinality 
    alfa = 0.7213/(1+1.079/(2**n))
    Z=0
    for i in range(len(buckets)):
        Z+=2**(-buckets[i]-1)
    Z=Z**(-1)
    E = alfa*(m)**2*Z 
    return (E)

### Cardinality of the hyperloglog:

The cardinality of the hyperloglog is given by:
${\displaystyle E=\alpha _{m}m^{2}Z}$ 


Where Z is: 
${\displaystyle Z={\Bigg (}\sum _{j=1}^{m}{2^{-M[j]}}{\Bigg )}^{-1}}$ 


and $\alpha _{m}$ (for $m>=128$) is: $0.7213/(1+1.079/(2^{m}))$

In [6]:
file = "hash.txt" 
card = hyperloglog(file)
print('Cardinality: ', card)

Cardinality:  121891579.94193806


### Error of the hyperloglog:
In order to see the error of our hyperloglog we can compare the value of distinct elements in the file hash.txt and the obtained cardinality.

In [14]:
real_len = 125000001 # obtained with set()
error = round(((real_len-card)/real_len)*100,2)
print('Error: ',error,'%')

Error:  2.49 %
